In [13]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

# 创建的C1是大小为1的所有候选项的集合
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:  # 遍历每一个项集
        for item in transaction:  # 遍历项集中的每一个项
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return list(map(frozenset, C1))

# 用于从C1生成L1（满足最小支持度的项集），以及包含支持度值的字典以备后用
# 参数：包含候选集合的列表D，数据集Ck，以及感兴趣集的最小支持度minSupport
# 返回值：保存满足最小支持度的数据列表retList，支持度字典
def scanD(D, Ck, minSupport):
    ssCnt = {}  # 记录每个数据在所有项集中出现的次数
    for tid in D:  # 遍历每一个项集
        for can in Ck:  # 遍历每一个数据
            if can.issubset(tid):  # 如果数据是项集i的子集，则该数据支持度+1
                if can not in ssCnt:
                    ssCnt[can] = 1
                else:
                    ssCnt[can] += 1
    numItems = float(len(D))  # 求项集数量
    retList = []
    supportData = {}
    for key in ssCnt:  # 遍历每一个数据
        support = ssCnt[key] / numItems  # 计算支持度
        if support >= minSupport:
            retList.insert(0, key)
        supportData[key] = support
    return retList, supportData

In [14]:
dataSet = loadDataSet()
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [15]:
C1 = createC1(dataSet)
C1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [16]:
# 求得不重复的候选集合列表
D = list(map(set, dataSet))
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [17]:
L1, suppData0  = scanD(D, C1, 0.5)
L1

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [67]:
# 创建候选集Ck
# 参数：频繁项集列表Lk，项集元素个数k
# 举例：该函数输入{0},{1},{2}，则会生成{0,1},{0,2},{1,2}（k=2）
def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
#     print(Lk, k)
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[:k-2]
#             print('L1: ', L1)
            L2 = list(Lk[j])[:k-2]
#             print('L2: ', L2)
            L1.sort()
            L2.sort()
            if L1 == L2:
                retList.append(Lk[i] | Lk[j])
#     print('retList: ', retList)
#     print('==================================================')
    return retList

# 返回频繁项集列表，支持度字典
def apriori(dataSet, minSupport=0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while len(L[k-2]) > 0:
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [68]:
L, suppData = apriori(dataSet)
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [69]:
L[0]

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [70]:
L[1]

[frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})]

In [71]:
L[2]

[frozenset({2, 3, 5})]

In [72]:
L[3]

[]

In [73]:
aprioriGen(L[0], 2)

[frozenset({2, 5}),
 frozenset({3, 5}),
 frozenset({1, 5}),
 frozenset({2, 3}),
 frozenset({1, 2}),
 frozenset({1, 3})]

In [74]:
L, suppData = apriori(dataSet, 0.7)
L

[[frozenset({5}), frozenset({2}), frozenset({3})], [frozenset({2, 5})], []]

In [90]:
# 参数：频繁项集列表，包含那些频繁项集支持数据的字典，最小可信度阈值
# 返回：包含可信度的规则列表
def generateRules(L, supportData, minConf=0.7):
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[i]:  # 从第2个（第1个都是单个元素没有意义）开始遍历频繁列表
#             print(freqSet)
            H1 = [frozenset([item]) for item in freqSet]  # 将freqSet每项的元素存为列表
#             print(H1)
            if i > 1:  # 如果频繁项超过2则计算其是否可继续分
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList

# 对规则进行评估
def calcConf(freqSet, H, supportData, br1, minConf=0.7):
    prunedH = []  # 记录通过评估的元素
    for conseq in H:
        conf = supportData[freqSet] / supportData[freqSet - conseq]  # 计算可信度
        if conf >= minConf:
#             print(freqSet, conseq)
#             print(freqSet - conseq, '-->', conseq, 'conf: ', conf)
            br1.append((freqSet - conseq, conseq, conf))  # 填充规则列表
            prunedH.append(conseq)
    return prunedH

# 生成候选规则集合
def rulesFromConseq(freqSet, H, supportData, br1, minConf=0.7):
    m = len(H[0])  # 每个频繁项的大小
    if len(freqSet) > m + 1:
        Hmp1 = aprioriGen(H, m+1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, br1, minConf)
        if len(Hmp1) > 1:
            rulesFromConseq(freqSet, Hmp1, supportData, br1, minConf)

In [88]:
L, suppData = apriori(dataSet, minSupport=0.5)
rules = generateRules(L, suppData, minConf=0.7)
rules

frozenset({5}) --> frozenset({2}) conf:  1.0
frozenset({2}) --> frozenset({5}) conf:  1.0
frozenset({1}) --> frozenset({3}) conf:  1.0


[(frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({1}), frozenset({3}), 1.0)]

In [89]:
rules = generateRules(L, suppData, minConf=0.5)
rules

frozenset({3}) --> frozenset({2}) conf:  0.6666666666666666
frozenset({2}) --> frozenset({3}) conf:  0.6666666666666666
frozenset({5}) --> frozenset({3}) conf:  0.6666666666666666
frozenset({3}) --> frozenset({5}) conf:  0.6666666666666666
frozenset({5}) --> frozenset({2}) conf:  1.0
frozenset({2}) --> frozenset({5}) conf:  1.0
frozenset({3}) --> frozenset({1}) conf:  0.6666666666666666
frozenset({1}) --> frozenset({3}) conf:  1.0
frozenset({5}) --> frozenset({2, 3}) conf:  0.6666666666666666
frozenset({3}) --> frozenset({2, 5}) conf:  0.6666666666666666
frozenset({2}) --> frozenset({3, 5}) conf:  0.6666666666666666


[(frozenset({3}), frozenset({2}), 0.6666666666666666),
 (frozenset({2}), frozenset({3}), 0.6666666666666666),
 (frozenset({5}), frozenset({3}), 0.6666666666666666),
 (frozenset({3}), frozenset({5}), 0.6666666666666666),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({3}), frozenset({1}), 0.6666666666666666),
 (frozenset({1}), frozenset({3}), 1.0),
 (frozenset({5}), frozenset({2, 3}), 0.6666666666666666),
 (frozenset({3}), frozenset({2, 5}), 0.6666666666666666),
 (frozenset({2}), frozenset({3, 5}), 0.6666666666666666)]

### 实例：发现毒蘑菇的相似特征

In [104]:
mushDatSet = [line.split() for line in open('mushroom.dat').readlines()]
mushDatSet[0]

['1',
 '3',
 '9',
 '13',
 '23',
 '25',
 '34',
 '36',
 '38',
 '40',
 '52',
 '54',
 '59',
 '63',
 '67',
 '76',
 '85',
 '86',
 '90',
 '93',
 '98',
 '107',
 '113']

In [105]:
L, suppData = apriori(mushDatSet, minSupport=0.3)
for item in L[1]:
    # intersection() 方法用于返回两个或更多集合中都包含的元素，即交集
    if item.intersection('2'):  
        print(item)

frozenset({'28', '2'})
frozenset({'2', '53'})
frozenset({'23', '2'})
frozenset({'34', '2'})
frozenset({'36', '2'})
frozenset({'59', '2'})
frozenset({'63', '2'})
frozenset({'67', '2'})
frozenset({'76', '2'})
frozenset({'85', '2'})
frozenset({'86', '2'})
frozenset({'90', '2'})
frozenset({'93', '2'})
frozenset({'39', '2'})


In [106]:
# 可以对更大的项集来观测
for item in L[3]:
    # intersection() 方法用于返回两个或更多集合中都包含的元素，即交集
    if item.intersection('2'):  
        print(item)

frozenset({'28', '34', '59', '2'})
frozenset({'28', '34', '85', '2'})
frozenset({'28', '34', '86', '2'})
frozenset({'28', '34', '2', '53'})
frozenset({'28', '34', '63', '2'})
frozenset({'28', '59', '63', '2'})
frozenset({'28', '85', '63', '2'})
frozenset({'28', '63', '86', '2'})
frozenset({'28', '85', '59', '2'})
frozenset({'28', '85', '86', '2'})
frozenset({'28', '2', '85', '53'})
frozenset({'28', '59', '86', '2'})
frozenset({'28', '2', '86', '53'})
frozenset({'90', '28', '53', '2'})
frozenset({'90', '28', '39', '2'})
frozenset({'90', '28', '34', '2'})
frozenset({'90', '28', '2', '59'})
frozenset({'90', '28', '85', '2'})
frozenset({'90', '28', '86', '2'})
frozenset({'28', '53', '39', '2'})
frozenset({'28', '34', '39', '2'})
frozenset({'28', '59', '39', '2'})
frozenset({'28', '39', '63', '2'})
frozenset({'28', '85', '39', '2'})
frozenset({'28', '39', '86', '2'})
frozenset({'34', '2', '85', '53'})
frozenset({'34', '2', '86', '53'})
frozenset({'2', '85', '86', '53'})
frozenset({'90', '2'